<a href="https://colab.research.google.com/github/yu072333/114-/blob/main/%E7%A8%8B%E8%A8%ADHW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Cell 1：安裝必要套件（第一次跑才需要）
!pip install -q -U google-generativeai pandas gspread gspread_dataframe

import google.generativeai as genai
print("google-generativeai 版本：", genai.__version__)


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_set_from_installed()
              

In [16]:
# Cell 2：匯入套件 & 設定 Gemini API

import pandas as pd
import datetime
import google.generativeai as genai

# 讀取 Colab 使用者金鑰（名稱：hw6）
try:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('gemini')
except ImportError:
    import os
    GEMINI_API_KEY = os.environ.get('gemini')

if GEMINI_API_KEY is None:
    raise ValueError("找不到名為 'hw6' 的 Gemini API key，請先在 Colab 設定。")

# 設定 API Key
genai.configure(api_key=GEMINI_API_KEY)

# 使用新版模型（建議）
model = genai.GenerativeModel("gemini-2.5-flash")
# 如果之後想省一點，可改成：
# model = genai.GenerativeModel("gemini-2.5-flash-lite")

print("✅ Gemini 設定完成（使用 gemini-2.5-flash）")


✅ Gemini 設定完成（使用 gemini-2.5-flash）


In [17]:
# Cell 3：Google 授權 & 讀取「整學期紀錄」分頁

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
from gspread_dataframe import get_as_dataframe, set_with_dataframe

creds, _ = default()
gc = gspread.authorize(creds)

# 你的試算表網址（裡面有「整學期紀錄」這個分頁）
sheet_url = "https://docs.google.com/spreadsheets/d/1fTjyuufHc1uGKebeCeOSy0LTfSdKT19NE6deH7yNOo0/edit?gid=1938721282#gid=1938721282"

# 開啟試算表
sh = gc.open_by_url(sheet_url)

# 分頁名稱：請確認跟你的工作表名稱一模一樣
worksheet_name = "整學期紀錄"

ws_main = sh.worksheet(worksheet_name)

# 轉成 DataFrame（第一列當欄位名稱）
df = get_as_dataframe(ws_main, evaluate_formulas=True)

# 移除全空列
df = df.dropna(how="all")

print("✅ 已讀取工作表：", worksheet_name)
df.head()

✅ 已讀取工作表： 整學期紀錄


,日期,星期,課程名稱,時間(起),時間(迄),教室,攜帶品,先讀章節,備註
0,2025-09-01,星期一,人力資源資訊系統,14:20,17:20,科技系TB311教室,NaN,NaN,NaN
1,2025-09-01,星期一,大師創業管理入門,18:40,20:25,誠201,NaN,NaN,NaN
2,2025-09-02,星期二,網際網路概論,09:10,12:10,科技系TB311教室,NaN,NaN,NaN
3,2025-09-03,星期三,多媒體教材設計與製作,14:20,17:20,科技系TC411教室,NaN,NaN,NaN
4,2025-09-04,星期四,程式語言,09:10,12:10,科技系TB311教室,NaN,NaN,NaN


In [18]:
# Cell 4：取得某一天的課程

def get_schedule_for_date(df, target_date=None):
    """
    df：整學期紀錄 DataFrame
    target_date：
        - None = 使用今天
        - "YYYY/MM/DD" 或 "YYYY-MM-DD"
    回傳：(date物件, 當天課程的 DataFrame)
    """
    if target_date is None:
        day = datetime.date.today()
    else:
        # 支援 "YYYY/MM/DD" 或 "YYYY-MM-DD"
        if "-" in target_date:
            day = datetime.date.fromisoformat(target_date)
        else:
            day = datetime.datetime.strptime(target_date, "%Y/%m/%d").date()

    # df["日期"] 轉成 date 型態來比對
    date_series = pd.to_datetime(df["日期"], errors="coerce").dt.date
    day_df = df[date_series == day].copy()

    return day, day_df

# 🔎 測試：抓開學日 2025/09/01 的課
test_day, test_day_df = get_schedule_for_date(df, "2025/09/01")
print("📅 測試日期：", test_day)
test_day_df


📅 測試日期： 2025-09-01


,日期,星期,課程名稱,時間(起),時間(迄),教室,攜帶品,先讀章節,備註
0,2025-09-01,星期一,人力資源資訊系統,14:20,17:20,科技系TB311教室,NaN,NaN,NaN
1,2025-09-01,星期一,大師創業管理入門,18:40,20:25,誠201,NaN,NaN,NaN


In [22]:
# Cell 5：為每一堂課產生 AI 行前提醒

def generate_reminder_for_row(row):
    course = row.get("課程名稱", "")
    start_time = row.get("時間(起)", "")
    end_time = row.get("時間(迄)", "")
    classroom = row.get("教室", "")

    prompt = f"""
你是一位溫暖又務實的 AI 學習助理。
請根據下面的課程資訊，幫學生寫一段 1～2 句的「行前提醒」，用繁體中文，40 字以內。

課程資訊：
- 課程名稱：{course}
- 上課時間：{start_time} ~ {end_time}
- 教室：{classroom}

提醒要求：
1. 點出這堂課可以預習什麼內容
2. 語氣輕鬆、帶有鼓勵，但不要太浮誇
3. 總結出一小段文字
"""

    response = model.generate_content(prompt)
    return response.text.strip()

def attach_reminders_for_date(df, target_date=None):
    """
    取得某一天課程，並加上「行前提醒」欄位
    """
    day, day_df = get_schedule_for_date(df, target_date)

    if len(day_df) == 0:
        print(f"{day} 這天沒有課程 🎉")
        return day, day_df

    day_df = day_df.copy()
    day_df["行前提醒"] = day_df.apply(generate_reminder_for_row, axis=1)
    return day, day_df

# 🔎 測試：開學日的行前提醒
test_day, reminder_test_df = attach_reminders_for_date(df, "2025/09/01")
reminder_test_df

KeyboardInterrupt: 

In [20]:
# Cell 6：把某一天的提醒組成一段文字（選用）

def build_daily_message(reminder_df, day_obj):
    if len(reminder_df) == 0:
        return f"{day_obj.strftime('%Y/%m/%d')} 今天沒有排課，休息一下也很重要喔 😌"

    weekday = reminder_df.iloc[0]["星期"]  # 例如 "一", "二"
    header = f"📚 {day_obj.strftime('%Y/%m/%d')}（星期{weekday}）課程提醒\n"

    lines = [header]
    for _, row in reminder_df.iterrows():
        course = row["課程名稱"]
        start_time = row["時間(起)"]
        end_time = row["時間(迄)"]
        classroom = row["教室"]
        reminder = row["行前提醒"]

        block = f"· {start_time}~{end_time}｜{course}（{classroom}）\n  👉 {reminder}"
        lines.append(block)

    return "\n".join(lines)

# 🔎 測試：開學日的訊息
demo_msg = build_daily_message(reminder_test_df, test_day)
print(demo_msg)

📚 2025/09/01（星期星期一）課程提醒

· 14:20~17:20｜人力資源資訊系統（科技系TB311教室）
  👉 這堂「人力資源資訊系統」，可預習HRIS的基本概念，或科技如何應用於人資，讓學習更順利喔！
· 18:40~20:25｜大師創業管理入門（誠201）
  👉 先想想你對創業的點子或好奇，上課會更有收穫喔！


In [21]:
# Cell 7：產生某一週的課程 + 行前提醒 DataFrame

def build_week_df_with_reminders(df, week_start_date_str):
    """
    week_start_date_str：這一週的起始日期（通常是週一），格式 "YYYY/MM/DD"
    回傳：該週所有有課的列，包含「行前提醒」欄位
    """
    week_start = datetime.datetime.strptime(week_start_date_str, "%Y/%m/%d").date()

    week_list = []

    for i in range(7):  # 七天
        day = week_start + datetime.timedelta(days=i)
        day_str = day.strftime("%Y/%m/%d")

        _, day_df = attach_reminders_for_date(df, day_str)
        if len(day_df) == 0:
            continue

        day_df = day_df.copy()
        # 加一個真正的日期欄位，方便排序
        day_df["日期_dt"] = pd.to_datetime(day_df["日期"], errors="coerce")

        week_list.append(day_df)

    if not week_list:
        # 沒有任何課的週
        return pd.DataFrame(columns=["日期", "星期", "課程名稱", "時間(起)", "時間(迄)", "教室", "行前提醒"])

    week_df = pd.concat(week_list, ignore_index=True)

    # 依日期、時間排序
    week_df = week_df.sort_values(by=["日期_dt", "時間(起)"])

    # 最後只保留需要欄位
    week_df = week_df[["日期", "星期", "課程名稱", "時間(起)", "時間(迄)", "教室", "行前提醒"]]

    return week_df

# 🔎 測試：產生「開學那一週」的週摘要
week_df_demo = build_week_df_with_reminders(df, "2025/09/01")
week_df_demo.head(20)


KeyboardInterrupt: 

In [23]:
# Cell 8：將週摘要回寫到 Google Sheet（自動建立分頁）

def write_week_summary_to_sheet(sh, df, week_start_date_str):
    """
    sh：gspread Spreadsheet 物件（這裡就是前面的 sh）
    df：整學期紀錄 DataFrame
    week_start_date_str："YYYY/MM/DD"，例如 "2025/09/01"
    """
    week_df = build_week_df_with_reminders(df, week_start_date_str)

    if week_df.empty:
        print(f"{week_start_date_str} 這週沒有課程，不建立分頁。")
        return

    # 分頁名稱：例如 "週摘要_2025-09-08"
    week_start = datetime.datetime.strptime(week_start_date_str, "%Y/%m/%d").date()
    sheet_name = f"週摘要_{week_start.strftime('%Y-%m-%d')}"

    # 檢查是否已經有同名分頁
    try:
        ws_week = sh.worksheet(sheet_name)
        ws_week.clear()
        print(f"🔁 已存在分頁「{sheet_name}」，清空後覆寫。")
    except gspread.WorksheetNotFound:
        ws_week = sh.add_worksheet(title=sheet_name, rows="200", cols="10")
        print(f"🆕 已新增分頁「{sheet_name}」。")

    # 寫入 DataFrame（含欄位名稱）
    set_with_dataframe(ws_week, week_df, include_index=False, include_column_header=True)

    print(f"✅ 已將 {week_start_date_str} 那一週的摘要寫入分頁：{sheet_name}")


# 🔎 測試：把「開學那一週」摘要寫回試算表
write_week_summary_to_sheet(sh, df, "2025/09/01")

KeyboardInterrupt: 

In [24]:
# Cell 9：Gradio 介面（使用最相容的 Textbox 日期輸入）

import gradio as gr
import datetime

DISPLAY_COLS = ["日期", "星期", "課程名稱", "時間(起)", "時間(迄)", "教室", "行前提醒"]

def _parse_date_input(text):
    """把使用者輸入的文字轉成 date；空白則使用今天"""
    if text is None or text.strip() == "":
        return datetime.date.today()

    text = text.strip()
    try:
        # 支援 YYYY-MM-DD
        return datetime.date.fromisoformat(text)
    except:
        try:
            # 支援 YYYY/MM/DD
            return datetime.datetime.strptime(text, "%Y/%m/%d").date()
        except:
            return datetime.date.today()


# === 每日模式 ===
def gr_generate_daily(date_text):
    day_obj = _parse_date_input(date_text)
    date_str = day_obj.strftime("%Y/%m/%d")

    day, day_df = attach_reminders_for_date(df, date_str)

    if len(day_df) == 0:
        msg = f"{date_str} 今天沒有排課 😌"
        empty_df = pd.DataFrame(columns=DISPLAY_COLS)
        return msg, empty_df

    msg = build_daily_message(day_df, day_obj)
    display_df = day_df[DISPLAY_COLS].reset_index(drop=True)
    return msg, display_df


# === 每週模式 ===
def gr_generate_week(date_text, write_to_sheet):
    day_obj = _parse_date_input(date_text)

    # 計算這一週的星期一
    monday = day_obj - datetime.timedelta(days=day_obj.weekday())
    week_start_str = monday.strftime("%Y/%m/%d")

    week_df = build_week_df_with_reminders(df, week_start_str)

    if week_df.empty:
        msg = f"{week_start_str} 那一週沒有任何課程 😆"
        empty_df = pd.DataFrame(columns=DISPLAY_COLS)
        return msg, empty_df

    # 組成一週的提醒文字
    all_msgs = []
    for date_str in sorted(week_df["日期"].unique()):
        sub = week_df[week_df["日期"] == date_str]
        d = datetime.datetime.strptime(date_str, "%Y/%m/%d").date()
        day_msg = build_daily_message(sub, d)
        all_msgs.append(day_msg)

    weekly_msg = f"📆 本週課程 AI 行前提醒（起始：{week_start_str}）\n\n"
    weekly_msg += "\n\n---\n\n".join(all_msgs)

    # 回寫試算表
    if write_to_sheet:
        write_week_summary_to_sheet(sh, df, week_start_str)
        weekly_msg += f"\n\n✅ 已寫入試算表：週摘要_{monday.strftime('%Y-%m-%d')}"

    display_df = week_df[DISPLAY_COLS].reset_index(drop=True)
    return weekly_msg, display_df


# === Gradio UI ===
with gr.Blocks(title="AI 個人化學習助理") as demo:
    gr.Markdown(
        """
# 🧠 AI 個人化學習助理
使用整學期課表 + Gemini，產生每日行前提醒與每週摘要。
（日期請輸入：2025-09-01 或留白＝今天）
        """
    )

    # ------- 每日 Tab -------
    with gr.Tab("📅 每日行前提醒"):
        date_input_daily = gr.Textbox(label="輸入日期（YYYY-MM-DD，不填=今天）")
        daily_btn = gr.Button("產生行前提醒", variant="primary")
        daily_msg = gr.Markdown()
        daily_table = gr.Dataframe(headers=DISPLAY_COLS, interactive=False)

        daily_btn.click(
            fn=gr_generate_daily,
            inputs=date_input_daily,
            outputs=[daily_msg, daily_table],
        )

    # ------- 每週 Tab -------
    with gr.Tab("📆 每週摘要"):
        date_input_week = gr.Textbox(label="輸入任一天（YYYY-MM-DD）")
        write_check = gr.Checkbox(label="回寫 Google Sheet（新增 / 更新 週摘要）")
        week_btn = gr.Button("產生本週摘要", variant="primary")
        week_msg = gr.Markdown()
        week_table = gr.Dataframe(headers=DISPLAY_COLS, interactive=False)

        week_btn.click(
            fn=gr_generate_week,
            inputs=[date_input_week, write_check],
            outputs=[week_msg, week_table],
        )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f2d3163c3e0596d796.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
